# Başlamadan Önce Chromdriver.exe'yi [Link](http://chromedriver.storage.googleapis.com/index.html?path=75.0.3770.8/) 'ten indirmeyi unutmayın!!




Yunus Emre Gündoğmuş - 2019 Mayıs

In [ ]:
# Kütüphanelerin Yüklenmesi
!pip install selenium
!pip install keyboard
!pip install pynput

### Kütüphanelerin İçeri Import Edilmesi

In [43]:
# Selenium
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

# Keyboard and Time
from pynput.keyboard import Key, Controller
import datetime 
import time
import keyboard
import numpy as np
import sys

# Keras and LSTM
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [134]:
# Notalarımızın Dictionarysi (Apronus'taki klavyedeki tuşlara göre)
notalar = {"Dm#":"d","Bm7":"6","F7":'"',"Ab":"w","G7":"1","C7":"2","D7":"3","E7":"4","A7":"5","B7":"6","C#7":"7","F#7":"8","G#7":"9","A#7":"0","D#7":"-",
           "Dm":"tab","Em":"q","Am":"w","Bm":"e","C#m":"r","F#m":"t","G#m":"y","A#m":"u","D#m":"ı","Fm":"o","Gm":"p","Cm":"ğ",
           "F":"capslock","G":"a","C":"s","D":"d","E":"f","A":"g","B":"h","C#":"j","F#":"k","G#":"l","A#":"ş","D#":"i"}


# Online Gitar için Klavyede Tuşa Basma Fonksiyonu
def Press(nota):
    keyboard = Controller()
    if nota == "Dm":
        keyboard.press(Key.tab)
    elif nota == "F":
        keyboard.press(Key.caps_lock)
    else:
        keyboard.press(notalar[nota])
        time.sleep(0.7)
        keyboard.release(notalar[nota])
        
# Selenium'un Ana Fonksiyonu
def Play(notalist):
    #Online Gitar Sitesinin Açılması
    driver = webdriver.Chrome('chromedriver.exe')
    driver.get("https://www.apronus.com/music/onlineguitar.htm");
    driver.implicitly_wait(10)
    
    time.sleep(5)
    # Eğer Notamız Sleep ise 0.5 Saniye bekliyoruz.
    for i in notalist:
        if i == "sleep":
            time.sleep(0.5)
        # Eğer Notamız New ise şarkı bitiyor.
        elif i == "new":
            break
        else:
            Press(i)

### Yeni notaları üretmemizi sağlayacak random kısmı kendi verisetimizden seçebilir veya kendimiz belirleyebiliriz. Ben burada verisetinden seçmek istediğim için verisetini içeri alıyorum.

In [99]:
# Txt Dosyasının Açılması
myfile = open("nirvananota.txt","r")
data = myfile.read().split("\n")

# Boşlukların temizlenmesi bütün notaların tek tek eleman olarak alınması
clean = []
for i in range(0,len(data)):
    clean = clean + [x for x in data[i].split(" ") if str(x) != '']

# Karakterler için Dictionary Oluşturulması
chars = sorted(list(set(clean)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
n_chars = len(clean)
n_vocab = len(chars)

# Notaların birbirlerine bağlanması örneğin ;
# E7 G E C --> E7 G E TARGET/C
seq_length = 10
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = clean[i:i + seq_length]
    seq_out = clean[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
int_to_char = dict((i, c) for i, c in enumerate(chars))

X = np.reshape(dataX, (n_patterns, seq_length, 1))
# Normalize Etmek.
X = X / float(n_vocab)
# Y değişkeni için One Hot Encoding Yapıyoruz.
y = np_utils.to_categorical(dataY)

### Burada Zaten Eğittiğim modeli çağırıyorum. Sadece mimarisini tanıtıp, weightlerini çağırıyor ve kullanıma hazır hale getiriyorum

In [100]:
model = Sequential()
model.add(LSTM(512, return_sequences=True,
                input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.5))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(512))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

filename = "stacked-weights-improvement-80-0.2467.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

### Bu kısımda olay başlıyor. Yapılan olay şu, öncelikle başlangıç değerleri veriliyor bunlara göre predictionlar yapılarak sürekli yeni nota tahminleniyor ardından sürekli başlangıç değerleri silinerek devam ediliyor. 

### Örnek
- D B7 D B7 A G F# sleep A F için bir nota tahmin edildi. Mesela buna **B7** diyelim. Ardından
- B7 D B7 A G F# sleep A F B7 şeklinde verilip yeni tahmin alınıyor.

In [153]:
yeninota = []
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Başlangıç Değerleri:")
print(''.join([int_to_char[value] for value in pattern]))
print("Çıktılarımız:")
# generate characters
for i in range(50):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in4 = [int_to_char[value] for value in pattern]
    sys.stdout.write(result+str(" "))
    pattern.append(index)
    yeninota.append(index)
    pattern = pattern[1:len(pattern)]
yeninota = yeninota[len(pattern):]

Başlangıç Değerleri:
AFB7AFB7AFB7sleep
Çıktılarımız:
E D B7 D B7 D B7 D B7 A G F# sleep A C F B7 sleep A C F B7 sleep A C F B7 sleep A C F B7 sleep A C F B7 sleep A C F B7 sleep A C F B7 sleep A C 

### Başlangıç Değerlerini Kendimiz Belirleyelim.

In [167]:
yeninota = []
pattern = [1,5,7,8,9,1,3,5,11,12]

print("Başlangıç Değerleri:")
print(''.join([int_to_char[value] for value in pattern]))
print("Çıktılarımız:")
# generate characters
for i in range(50):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in4 = [int_to_char[value] for value in pattern]
    sys.stdout.write(result+str(" "))
    pattern.append(index)
    yeninota.append(index)
    pattern = pattern[1:len(pattern)]
yeninota = yeninota[len(pattern):]

Başlangıç Değerleri:
A#AmB7Bm7CA#A7AmC#7C#m
Çıktılarımız:
A# E7 C A# C B A# sleep C7 G F# G#7 E7 G7 E7 G7 G#7 G#7 G7 E7 G7 E7 G7 B7 G#7 E7 G7 E7 G7 B7 G#7 E7 G7 E7 G7 B7 G#7 E7 G7 E7 G7 B7 G#7 E7 G7 E7 G7 B7 G#7 E7 

### Yeni notalar sayı halinde geldiği için geri nota şekline çeviriyoruz

In [168]:
yeninotalar =[]
for i in yeninota:
    yeninotalar.append(int_to_char[i])
    
print(yeninotalar)

['F#', 'G#7', 'E7', 'G7', 'E7', 'G7', 'G#7', 'G#7', 'G7', 'E7', 'G7', 'E7', 'G7', 'B7', 'G#7', 'E7', 'G7', 'E7', 'G7', 'B7', 'G#7', 'E7', 'G7', 'E7', 'G7', 'B7', 'G#7', 'E7', 'G7', 'E7', 'G7', 'B7', 'G#7', 'E7', 'G7', 'E7', 'G7', 'B7', 'G#7', 'E7']


### Selenium'a notaları verip çaldırmaya başlıyoruz.

In [169]:
Play(yeninotalar)